Lecture: AI I - Basics 

Previous:
[**Chapter 4.2: Machine Learning with scikit-learn**](../04_ml/02_machine_learning.ipynb)

---

# Chapter 4.3: Evaluation with scikit-learn

- [Cross-Validation](#cross-validation)
- [Hyperparameter Tuning](#hyperparameter-tuning)
- [Metrics](#metrics)
- [Visualization of Results](#visualization-of-results)

Model selection refers to choosing the ML model that best explains the given data. However, since there is a wide variety of different models, each with large parameter ranges, this task is very critical. For example, if a model is selected that performs very well on the training data but fails miserably on new data, it has not learned the underlying data distribution.  

In this notebook, we will cover the following topics:  
- Cross-validation  
- Hyperparameter tuning  
- Metrics and results  
- Visualization of results  

The topic of model selection and evaluation is also covered in great detail in the [sklearn user guide](https://scikit-learn.org/stable/model_selection.html).  


In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets

---

Lecture: AI I - Basics 

Exercise: [**Exercise 4.3: Evaluation with scikit-learn**](../04_ml/exercises/03_evaluation.ipynb)

Next: [**Chapter 5.1: Assesment 1**]()